<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/9_%EA%B8%B0%EB%B3%B8%EC%BF%BC%EB%A6%AC_%EB%B0%8F_%ED%95%A8%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL 기본개념
```
sql : 데이터 베이스를 정의하고 조작하는 언어, 질의어
주요명령어
    DDL(Data Definition Laguage) : create, alter,drop 테이블 생성 수정 삭제
    DML(Data Manipulation Laguage) : insert select update delete  데이터 CRUD
    DCL(Data Control Laguage) : grant, revoke  권한부여 삭제
함수 : 데이터 가공 및 계산을 위한 내장 함수 : count avg json_extract 등등..
```

# workbench에서 생성한 데이터베이스, 테이블에 데이터를 1000개 임의로 생성하기(프로그램 사용)

In [ ]:
!pip install mysql-connector-python -q

In [ ]:
from mysql import connector
# 접속 관련 매개변수 설정
args = {
    "host" : "localhost",
    "user" : "root",
    "password" : "root1234",
    "port" : 3306,
    "database" : "student_db2"
}
conn = connector.connect(**args)  # localhost 에 접속 해서 접속 객체를 생성
cursor = conn.cursor()
sql = """
insert into student_db2.students(name,age,kor,eng,math)
select
	concat(
		ELT(floor(1+rand()*8), '김','이','박','최','구','강','조','윤'),
        ELT(floor(1+rand()*5), '영희','철수','준호','철호','미나')
    ) as name,
    floor(20+rand()*20) as age,
    floor(70+rand()*21) as kor,
    floor(70+rand()*21) as eng,
    floor(70+rand()*21) as math
"""
for _ in range(1000):
    cursor.execute(sql)
    conn.commit()

cursor.close()
conn.close()

# 학생 테이블을 관리할 클래스
```
    DB 연결하고 닫는 기본적인 StudentMng 클래스 생성
```

In [ ]:
from mysql.connector import connect, Error

In [ ]:
# 생성자 매개변수 : host, user, password, databases
class StudentMng:
    def __init__(self, host='localhost', user='root',password='root1234',database='student_db2'):
        self.conn = None
        self.cursor = None
        self.args = { 'host':host, 'user':user, 'password':password,'database':database}
    def connection(self):
        try:
            self.conn = connector.connect(**self.args)
            self.cursor = self.conn.cursor()
            print(f'접속 성공')
        except Exception as e:
            print(f'연결 오류: {e}')
    def disconnect(self):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print('db 연결이 종료되었습니다')
    # 데이터추가 인스턴스 메소드
    # 모든 학생 조회 인스턴스 메소드
    # 학생정보 수정 인스턴스 메소드
    # 학생 삭제 인스턴스 메소드

In [ ]:
if __name__ == '__main__':
    # 객체 생성하고 db 연결한다음, 접속해제
    mngStudent = StudentMng()
    # 데이터베이스 접속
    mngStudent.connection()
    # 데이터 베이스 해제
    mngStudent.disconnect()

```
데이터추가 인스턴스 메소드
모든 학생 조회 인스턴스 메소드
학생정보 수정 인스턴스 메소드
학생 삭제 인스턴스 메소드
```

In [ ]:
# 생성자 매개변수 : host, user, password, databases
class StudentMng:
    def __init__(self, host='localhost', user='root',password='root1234',database='student_db2'):
        self.conn = None
        self.cursor = None
        self.args = { 'host':host, 'user':user, 'password':password,'database':database}
    def connection(self):
        try:
            self.conn = connect(**self.args)
            self.cursor = self.conn.cursor()
            print(f'접속 성공')
        except Exception as e:
            print(f'연결 오류: {e}')
    def disconnect(self):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print('db 연결이 종료되었습니다')
    # 학생데이터 생성
    def create_student(self,name,age,kor,eng,math):
        try:
            query = """
                insert into students values(null, %s, %s, %s, %s, %s)
            """
            self.cursor.execute(query,(name,age,kor,eng,math) )
            self.conn.commit()
            print('학생데이터 생성 완료')
        except Error as e:
            print(f"학생데이터 생성 오류 {e}")
    # 모든 학생테이블 조회
    def get_all_student(self):
        try:
            query = """
                select * from students
            """
            self.cursor.execute(query)
            results = self.cursor.fetchall()
            return results
        except Error as e:
            print(f"학생데이터 조회 오류 {e}")
    def update_student(self,id, name=None,age=None,kor=None,eng=None,math=None):
        try:
            query = """
                update students
                    set name = coalesce(%s,name),
                    age = coalesce(%s,age),
                    kor = coalesce(%s,kor),
                    eng = coalesce(%s,eng),
                    math = coalesce(%s,math)
                where id = %s
            """
            self.cursor.execute(query,(name,age,kor,eng,math,id))
            self.conn.commit()
        except Error as e:
            print(f"학생데이터 업데이트 오류 {e}")
    def delte_student(self,id):
        try:
            query = """
                delete from students where id = %s
            """
            self.cursor.execute(query, (id,) )
            self.conn.commit()
        except Error as e:
            print(f"학생데이터 삭제 오류 {e}")

In [ ]:
if __name__ == '__main__':
    # 객체생성
    s = StudentMng();
    s.connection()
    # 학생데이터 생성
    s.create_student('홍길동',5,15,25,35)
    print('조회----------------------------------')
    students = s.get_all_student()
    for idx, data in enumerate(students):
        if idx == 5:
            break
        print(data)
    print('업데이트----------------------------------')
    s.update_student(2,eng = 100)
    print('업데이트 후 데이터 확인')
    students = s.get_all_student()
    for idx, data in enumerate(students):
        if idx == 5:
            break
        print(data)
    # 삭제

#Streamlit 실행

In [ ]:
%%writefile app.py
import streamlit as st
from mysql.connector import connect, Error

class StudentMng:
    def __init__(self, host='localhost', user='root',password='root1234',database='student_db2'):
        self.conn = None
        self.cursor = None
        self.args = { 'host':host, 'user':user, 'password':password,'database':database}
    def connection(self):
        try:
            self.conn = connect(**self.args)
            self.cursor = self.conn.cursor()
            print(f'접속 성공')
        except Exception as e:
            print(f'연결 오류: {e}')
    def disconnect(self):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print('db 연결이 종료되었습니다')
    # 학생데이터 생성
    def create_student(self,name,age,kor,eng,math):
        try:
            query = """
                insert into students values(null, %s, %s, %s, %s, %s)
            """
            self.cursor.execute(query,(name,age,kor,eng,math) )
            self.conn.commit()
            print('학생데이터 생성 완료')
        except Error as e:
            print(f"학생데이터 생성 오류 {e}")
    # 모든 학생테이블 조회
    def get_all_student(self):
        try:
            query = """
                select * from students
            """
            self.cursor.execute(query)
            results = self.cursor.fetchall()
            return results
        except Error as e:
            print(f"학생데이터 조회 오류 {e}")
    def update_student(self,id, name=None,age=None,kor=None,eng=None,math=None):
        try:
            query = """
                update students
                    set name = coalesce(%s,name),
                    age = coalesce(%s,age),
                    kor = coalesce(%s,kor),
                    eng = coalesce(%s,eng),
                    math = coalesce(%s,math)
                where id = %s
            """
            self.cursor.execute(query,(name,age,kor,eng,math,id))
            self.conn.commit()
        except Error as e:
            print(f"학생데이터 업데이트 오류 {e}")
    def delte_student(self,id):
        try:
            query = """
                delete from students where id = %s
            """
            self.cursor.execute(query, (id,) )
            self.conn.commit()
        except Error as e:
            print(f"학생데이터 삭제 오류 {e}")

def main():
    st.title('학생관리 시스템')
    # 새션 상태로 Db 연결 유지
    if 'student_mng' not in st.session_state:
        st.session_state['student_mng'] = StudentMng()
        st.session_state['student_mng'].connection()
    student_mng = st.session_state['student_mng']

    # '사이드바 추가'
    menu = st.sidebar.selectbox('메뉴',['홈','학생추가','학생조회','학생수정','학생삭제'])
    if menu == '홈':
        st.write('안녕하세요! 학생관리 시스템에 오신것을 환영합니다.')
    else:
        st.write(f'{menu}기능 개발중이에요...')
    if st.sidebar.button('DB 연결 종료'):
        student_mng.disconnect()
        del st.session_state['student_mng']
        st.write('db 연결을 종료합니다.')

if __name__=='__main__':
    main()

In [ ]:
!streamlit run app.py